Copyright 2024 Zhixuan Hu

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [2]:
import numpy as np
import pandas as pd
import os, re, sys, json, time
sys.path.append(os.getcwd() + '\\..\\')

In [3]:
pattern = re.compile(r".*\.csv")
files = os.listdir('../../dataset/')
files = [file for file in files if pattern.match(file)]
files

['chatgpt_HC3_all.csv',
 'gpt_0125_00.csv',
 'human_00.csv',
 'human_01_01.csv',
 'human_01_02.csv',
 'human_02.csv',
 'human_cook_01.csv',
 'human_cook_02.csv',
 'human_HC3_all.csv',
 'human_modern_01.csv',
 'human_modern_02.csv',
 'human_travel_01.csv',
 'human_travel_02.csv',
 'human_travel_03.csv',
 'questions_00.csv',
 'questions_01_01.csv',
 'questions_01_02.csv',
 'questions_02.csv',
 'questions_cook_01.csv',
 'questions_cook_02.csv',
 'questions_HC3_all.csv',
 'questions_modern_01.csv',
 'questions_modern_02.csv',
 'questions_travel_01.csv',
 'questions_travel_02.csv',
 'questions_travel_03.csv',
 'test_gpt_0125.csv',
 'test_human.csv',
 'test_questions.csv']

In [4]:
file = '../../dataset/human_HC3_all.csv'
df = pd.read_csv(file)
df

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,question_id,text,answer_id,text-embedding-3-small
0,0,0,0,0,0,10042,改善型二套房可以。,1,"[0.0026415179017931223, 0.02066306211054325, 0..."
1,1,1,1,1,1,11849,我和我老公裸婚的，为了给父母聘礼，还借了几万块外债。然后我们一起出国打工，现在房子买了，有两...,1,"[0.041730642318725586, 0.022379141300916672, -..."
2,2,2,2,2,2,4103,《通信电子线路》是2010年5月北京理工大学出版社出版的图书，作者是夏术泉等。 \n《通信电...,1,"[0.0074668824672698975, -0.000739583105314523,..."
3,3,3,3,3,3,8815,除序言以外，共七章六十条款。,1,"[0.017011862248182297, 0.04433589056134224, 0...."
4,4,4,4,4,4,9394,第一任：于维鲁先生(57.9~64.8),1,"[0.03814779967069626, 0.0015381427947431803, 0..."
...,...,...,...,...,...,...,...,...,...
22211,22238,22238,22238,22238,22238,12405,似乎这个世界对我们来说是很危险的，尤其是当变化来临的时候，我们多么希望这个世界最好不变化，只...,6,"[0.05099976062774658, 0.014297748915851116, -0..."
22212,22239,22239,22239,22239,22239,4612,调制在通信系统中的作用至关重要。广义的调制分为基带调制和带通调制（也称载波调制）。在无线通信...,1,"[0.03942026197910309, 0.010326371528208256, -0..."
22213,22240,22240,22240,22240,22240,1835,这个要看你喜欢的姑娘风格。文艺风：河南省博物院，郑州市科技馆中原图书大厦河南省博物院： 从原...,3,"[0.025181010365486145, -0.026714330539107323, ..."
22214,22241,22241,22241,22241,22241,11430,之前发生过什么特别的事情呢？感觉你是缺乏安全感，特别是与父母的关系，安全感来源于原生家庭，这...,8,"[0.06056252121925354, -0.00241345027461648, 0...."


In [5]:
model = 'sentences_length[mean1,cv1,mean2,cv2]'
# 检查是否存在 'as' 列
if model not in df.columns:
    # 如果不存在，则创建一个空的字符串列
    df[model] = ''
df[model] = df[model].astype(str)

In [6]:
df_need = df[('"nan"' == df[model]) | (df[model]=='')]
# 防止过长超出token限制
tol = 1e20
mask = df_need['text'].str.len() > tol
df_need.loc[mask, 'text'] = df_need.loc[mask, 'text'].str.slice(0, tol) #截断
df_need_list = df_need['text'].to_list()
df_need

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,question_id,text,answer_id,text-embedding-3-small,"sentences_length[mean1,cv1,mean2,cv2]"
0,0,0,0,0,0,10042,改善型二套房可以。,1,"[0.0026415179017931223, 0.02066306211054325, 0...",
1,1,1,1,1,1,11849,我和我老公裸婚的，为了给父母聘礼，还借了几万块外债。然后我们一起出国打工，现在房子买了，有两...,1,"[0.041730642318725586, 0.022379141300916672, -...",
2,2,2,2,2,2,4103,《通信电子线路》是2010年5月北京理工大学出版社出版的图书，作者是夏术泉等。 \n《通信电...,1,"[0.0074668824672698975, -0.000739583105314523,...",
3,3,3,3,3,3,8815,除序言以外，共七章六十条款。,1,"[0.017011862248182297, 0.04433589056134224, 0....",
4,4,4,4,4,4,9394,第一任：于维鲁先生(57.9~64.8),1,"[0.03814779967069626, 0.0015381427947431803, 0...",
...,...,...,...,...,...,...,...,...,...,...
22211,22238,22238,22238,22238,22238,12405,似乎这个世界对我们来说是很危险的，尤其是当变化来临的时候，我们多么希望这个世界最好不变化，只...,6,"[0.05099976062774658, 0.014297748915851116, -0...",
22212,22239,22239,22239,22239,22239,4612,调制在通信系统中的作用至关重要。广义的调制分为基带调制和带通调制（也称载波调制）。在无线通信...,1,"[0.03942026197910309, 0.010326371528208256, -0...",
22213,22240,22240,22240,22240,22240,1835,这个要看你喜欢的姑娘风格。文艺风：河南省博物院，郑州市科技馆中原图书大厦河南省博物院： 从原...,3,"[0.025181010365486145, -0.026714330539107323, ...",
22214,22241,22241,22241,22241,22241,11430,之前发生过什么特别的事情呢？感觉你是缺乏安全感，特别是与父母的关系，安全感来源于原生家庭，这...,8,"[0.06056252121925354, -0.00241345027461648, 0....",


In [7]:
from sentences_length import main
df_need[model] = df_need['text'].apply(lambda row: json.dumps(main(row)))

In [8]:
indexer = df_need.index
df.loc[indexer] = df_need
df[df.columns[6:]]

,text,answer_id,text-embedding-3-small,"sentences_length[mean1,cv1,mean2,cv2]"
0,改善型二套房可以。,1,"[0.0026415179017931223, 0.02066306211054325, 0...","[8.0, 0.0, 8.0, 0.0]"
1,我和我老公裸婚的，为了给父母聘礼，还借了几万块外债。然后我们一起出国打工，现在房子买了，有两...,1,"[0.041730642318725586, 0.022379141300916672, -...","[37.333333333333336, 0.4116836386928037, 7.214..."
2,《通信电子线路》是2010年5月北京理工大学出版社出版的图书，作者是夏术泉等。 \n《通信电...,1,"[0.0074668824672698975, -0.000739583105314523,...","[38.90909090909091, 0.9144920701424335, 17.291..."
3,除序言以外，共七章六十条款。,1,"[0.017011862248182297, 0.04433589056134224, 0....","[13.0, 0.0, 6.0, 0.16666666666666666]"
4,第一任：于维鲁先生(57.9~64.8),1,"[0.03814779967069626, 0.0015381427947431803, 0...","[6.0, 0.7200822998230957, 6.0, 0.7200822998230..."
...,...,...,...,...
22211,似乎这个世界对我们来说是很危险的，尤其是当变化来临的时候，我们多么希望这个世界最好不变化，只...,6,"[0.05099976062774658, 0.014297748915851116, -0...","[59.0, 0.03389830508474576, 12.333333333333334..."
22212,调制在通信系统中的作用至关重要。广义的调制分为基带调制和带通调制（也称载波调制）。在无线通信...,1,"[0.03942026197910309, 0.010326371528208256, -0...","[34.0, 0.830849817166208, 18.09090909090909, 0..."
22213,这个要看你喜欢的姑娘风格。文艺风：河南省博物院，郑州市科技馆中原图书大厦河南省博物院： 从原...,3,"[0.025181010365486145, -0.026714330539107323, ...","[97.0, 0.8762886597938144, 14.076923076923077,..."
22214,之前发生过什么特别的事情呢？感觉你是缺乏安全感，特别是与父母的关系，安全感来源于原生家庭，这...,8,"[0.06056252121925354, -0.00241345027461648, 0....","[33.5, 0.6119402985074627, 10.5, 0.14285714285..."


In [9]:
time_str = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())
os.rename(file, '../../dataset/history/' + file + '-' + time_str + '.csv')
df.to_csv(file, encoding='utf-8-sig')